`PAIR PROGRAMMING INTRO ML`

Los objetivos de hoy son:
Buscar un conjunto de datos a analizar:
Es necesario que el conjunto de datos a analizar tenga variables numéricas y categóricas, primando que haya más de una variable de tipo numérico. Ya que tambien se utilizarán serán útiles a la hora de realizar predicciones en las clases de regresión lineal múltiple. Así mismo se recomienda que los datasets esten ligeramente limpios para facilidar los análisis.


In [45]:
import pandas as pd
import numpy as np
import sidetable
from datetime import datetime, timedelta


In [43]:
df= pd.read_csv('international_matches.csv')
df.sample(2)

,date,home_team,away_team,home_team_continent,away_team_continent,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,...,shoot_out,home_team_result,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
13483,2010-10-10,Congo,Eswatini,Africa,Africa,117,142,0,0,3,...,No,Win,67.0,NaN,68.0,69.0,69.0,NaN,NaN,NaN
16074,2013-06-15,Morocco,Gambia,Africa,Africa,77,162,462,137,2,...,No,Win,71.0,NaN,73.5,76.7,79.0,61.5,59.7,66.2


Explicar los datos y las variables disponibles en el conjunto de datos seleccionado:
Esto se hace para que comprendais mejor los datos con los que estais trabajando.
Realizar un EDA sencillo para la variable dependiente y la variable independiente.

EDA

Primero vamos a ver que variables categoricas tenemos

In [28]:
categoricas= df.select_dtypes(include= 'O')
categoricas.sample(2)

,date,home_team,away_team,home_team_continent,away_team_continent,tournament,city,country,shoot_out,home_team_result
4176,2000-02-02,Finland,Iceland,Europe,Europe,Nordic Championship,Cartagena,Spain,No,Lose
16142,2013-07-16,Namibia,Mozambique,Africa,Africa,COSAFA Cup,Kitwe,Zambia,No,Lose


In [49]:
# vamos a transformar date a tipo fecha
df['date'].apply(pd.to_datetime)

0       1993-08-08
1       1993-08-08
2       1993-08-08
3       1993-08-08
4       1993-08-08
           ...    
23916   2022-06-14
23917   2022-06-14
23918   2022-06-14
23919   2022-06-14
23920   2022-06-14
Name: date, Length: 23921, dtype: datetime64[ns]

Vamos a ver que variables numericas tenemos:

In [36]:
numericas= df.select_dtypes(include= ('int64', 'float64'))
numericas.sample(2)

,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
15905,11,38,1037,640,4,1,70.0,72.0,68.2,74.7,76.8,73.5,81.7,75.0
19627,127,114,247,286,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df.stb.missing()

,missing,total,percent
away_team_mean_defense_score,16357,23921,68.379248
home_team_mean_defense_score,16134,23921,67.447013
away_team_mean_midfield_score,15942,23921,66.644371
away_team_goalkeeper_score,15826,23921,66.159441
home_team_mean_midfield_score,15759,23921,65.879353
away_team_mean_offense_score,15609,23921,65.252289
home_team_goalkeeper_score,15542,23921,64.972200
home_team_mean_offense_score,15411,23921,64.424564
country,0,23921,0.000000
home_team_result,0,23921,0.000000


In [39]:
df.describe()

,home_team_fifa_rank,away_team_fifa_rank,home_team_total_fifa_points,away_team_total_fifa_points,home_team_score,away_team_score,home_team_goalkeeper_score,away_team_goalkeeper_score,home_team_mean_defense_score,home_team_mean_offense_score,home_team_mean_midfield_score,away_team_mean_defense_score,away_team_mean_offense_score,away_team_mean_midfield_score
count,23921.000000,23921.000000,23921.000000,23921.000000,23921.000000,23921.000000,8379.000000,8095.000000,7787.000000,8510.000000,8162.000000,7564.000000,8312.000000,7979.000000
mean,77.854688,80.797375,323.401488,315.453576,1.609214,1.068266,74.963838,74.212477,74.903249,75.818743,75.889292,74.424379,75.420019,75.259143
std,52.355225,53.232902,500.825725,490.944273,1.630127,1.263944,8.212242,8.225919,6.003114,6.268416,6.053110,5.937425,6.201906,6.124573
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,47.000000,47.000000,52.800000,53.300000,54.200000,52.800000,53.300000,54.200000
25%,33.000000,36.000000,0.000000,0.000000,0.000000,0.000000,70.000000,69.000000,71.000000,71.700000,72.500000,70.500000,71.300000,71.800000
50%,71.000000,73.000000,0.000000,0.000000,1.000000,1.000000,75.000000,74.000000,75.200000,75.700000,76.200000,74.500000,75.300000,75.500000
75%,115.000000,119.000000,547.000000,523.000000,2.000000,2.000000,81.000000,80.000000,78.800000,80.000000,79.500000,78.200000,79.700000,79.000000
max,211.000000,211.000000,2164.000000,2164.000000,31.000000,21.000000,97.000000,97.000000,91.800000,93.000000,93.200000,91.800000,93.000000,93.200000


In [40]:
df.shape

(23921, 25)

In [41]:
df.dtypes

date                              object
home_team                         object
away_team                         object
home_team_continent               object
away_team_continent               object
home_team_fifa_rank                int64
away_team_fifa_rank                int64
home_team_total_fifa_points        int64
away_team_total_fifa_points        int64
home_team_score                    int64
away_team_score                    int64
tournament                        object
city                              object
country                           object
neutral_location                    bool
shoot_out                         object
home_team_result                  object
home_team_goalkeeper_score       float64
away_team_goalkeeper_score       float64
home_team_mean_defense_score     float64
home_team_mean_offense_score     float64
home_team_mean_midfield_score    float64
away_team_mean_defense_score     float64
away_team_mean_offense_score     float64
away_team_mean_m

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23921 entries, 0 to 23920
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   date                           23921 non-null  object 
 1   home_team                      23921 non-null  object 
 2   away_team                      23921 non-null  object 
 3   home_team_continent            23921 non-null  object 
 4   away_team_continent            23921 non-null  object 
 5   home_team_fifa_rank            23921 non-null  int64  
 6   away_team_fifa_rank            23921 non-null  int64  
 7   home_team_total_fifa_points    23921 non-null  int64  
 8   away_team_total_fifa_points    23921 non-null  int64  
 9   home_team_score                23921 non-null  int64  
 10  away_team_score                23921 non-null  int64  
 11  tournament                     23921 non-null  object 
 12  city                           23921 non-null 

In [55]:
df.home_team_goalkeeper_score.unique()

array([nan, 93., 94., 83., 75., 82., 77., 73., 85., 90., 78., 91., 97.,
       63., 68., 84., 53., 74., 71., 89., 65., 66., 81., 49., 62., 72.,
       79., 64., 70., 80., 86., 88., 59., 76., 51., 87., 95., 69., 67.,
       61., 92., 52., 47., 54., 50., 55., 58., 60., 48., 56., 57.])

In [57]:
df.home_team_continent.unique()

array(['South America', 'Africa', 'Europe', 'Oceania', 'Asia',
       'North America'], dtype=object)

In [56]:
df.columns

Index(['date', 'home_team', 'away_team', 'home_team_continent',
       'away_team_continent', 'home_team_fifa_rank', 'away_team_fifa_rank',
       'home_team_total_fifa_points', 'away_team_total_fifa_points',
       'home_team_score', 'away_team_score', 'tournament', 'city', 'country',
       'neutral_location', 'shoot_out', 'home_team_result',
       'home_team_goalkeeper_score', 'away_team_goalkeeper_score',
       'home_team_mean_defense_score', 'home_team_mean_offense_score',
       'home_team_mean_midfield_score', 'away_team_mean_defense_score',
       'away_team_mean_offense_score', 'away_team_mean_midfield_score'],
      dtype='object')